# Dashboard
> Modular pieces for streamlit dashboards
> All of this is meant to be run in a streamlit environment and is likely to fail elsewhere

In [ ]:
#| default_exp dashboard

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| exporti
import json, os, inspect
import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd
import polars as pl
import datetime as dt

from typing import List, Tuple, Dict, Union, Optional

import altair as alt
import s3fs

from salk_toolkit.utils import *
from salk_toolkit.io import *
from salk_toolkit.pp import e2e_plot

import streamlit as st
from streamlit_option_menu import option_menu
from streamlit_dimensions import st_dimensions
import streamlit_authenticator as stauth

In [ ]:
#| export

# This is a horrible workaround to get faceting to work with altair geoplots that do not play well with streamlit
# See https://github.com/altair-viz/altair/issues/2369 -> https://github.com/vega/vega-lite/issues/3729

# Draw a matrix of plots using separate plots and st columns
def draw_plot_matrix(pmat,matrix_form = False):
    if not matrix_form: pmat = [[pmat]]
    cols = st.columns(len(pmat[0]))
    for j,c in enumerate(cols):
        for i, row in enumerate(pmat):
            c.altair_chart(pmat[i][j])

# Draw the plot described by pp_desc 
def st_plot(pp_desc,**kwargs):
    matrix_form = (pp_desc['plot'] == 'geoplot')
    plots = e2e_plot(pp_desc, return_matrix_of_plots=matrix_form, **kwargs)
    draw_plot_matrix(plots, matrix_form=matrix_form)

In [ ]:
#| export
def get_plot_width(key):
    wobj = st_dimensions(key=key) or { 'width': 900 }# Can return none so handle that
    return int(0.85*wobj['width']) # Needs to be adjusted down  to leave margin for plots

In [ ]:
#| export

# Open either a local or an s3 file
def open_fn(fname, *args, s3_fs=None, **kwargs):
    if fname[:3] == 's3:':
        if s3_fs is None: s3_fs = s3fs.S3FileSystem(anon=False)
        return s3_fs.open(fname,*args,**kwargs)
    else:
        return open(fname,*args,**kwargs)

In [ ]:
#| export

# ttl=None - never expire. Makes sense for potentially big data files
@st.cache_resource(show_spinner=False,ttl=None)
def read_annotated_data_cached(data_source,**kwargs):
    return read_annotated_data(data_source,**kwargs)


# This is cached very short term (1 minute) to avoid downloading it on every page change
# while still allowing users to be added / changed relatively responsively
@st.cache_resource(show_spinner=False,ttl=60)
def load_json_cached(fname, _s3_fs=None, **kwargs):
    with open_fn(fname,'r',s3_fs=_s3_fs) as jf:
        return json.load(jf)

# For saving json back 
def save_json(d, fname, _s3_fs=None, **kwargs):
    with open_fn(fname,'w',s3_fs=_s3_fs) as jf:
        json.dump(d,jf)
        
def alias_file(fname, file_map):
    if fname[:3]!='s3:' and fname in file_map and not os.path.exists(fname):
        #print(f"Redirecting {fname} to {file_map[fname]}")
        return file_map[fname]
    else: return fname

In [ ]:
#| export

# Main dashboard wrapper - WIP
class SalkDashboardBuilder:

    def __init__(self, data_source, auth_conf, public=False):
        
        # Allow deployment.json to redirect files from local to s3 if local missing (i.e. in deployment scenario)
        if os.path.exists('deployment.json'):
            dep_meta = load_json_cached('deployment.json')
            filemap = vod(dep_meta,'files',{})
            data_source = alias_file(data_source,filemap)
            auth_conf = alias_file(auth_conf,filemap)
        
        self.s3fs = s3fs.S3FileSystem(anon=False) # Initialize s3 access. Key in secrets.toml
        self.data_source = data_source
        self.public = public
        self.pages = []
        self.sb_info = st.sidebar.empty()
        
        # Load data
        with st.spinner("Loading data..."):
            self.df, self.meta = read_annotated_data_cached(data_source)
        
        # Set up authentication
        with st.spinner("Setting up authentication..."):
            config = load_json_cached(auth_conf, _s3_fs = self.s3fs)
            self.auth_conf_data, self.auth_conf_file = config, auth_conf
            self.auth = stauth.Authenticate(
                config['credentials'],
                config['cookie']['name'],
                config['cookie']['key'],
                config['cookie']['expiry_days'],
                config['preauthorized']
            )
            self.users = config['credentials']['usernames']

        if not public:
            if st.session_state["authentication_status"] is False:
                st.error('Username/password is incorrect')
            if st.session_state["authentication_status"] is None:
                st.warning('Please enter your username and password')
            self.auth.login('Login', 'main')

        uname = st.session_state['username']
        self.user = {'name': st.session_state['name'], 
                     'username': uname,
                     **self.users[uname] } if st.session_state["authentication_status"] else {}
        
    def save_auth_conf(self):
        with open_fn(self.auth_conf_file,'w',s3_fs=self.s3fs) as jf:
            json.dump(self.auth_conf_data,jf)

    # pos_id is for plot_width to work in columns
    def plot(self, pp_desc, pos_id=None, **kwargs):
        st_plot(pp_desc,
                width=min(get_plot_width(pos_id or 'full'),800),
                full_df=self.df,data_meta=self.meta,**kwargs)

    def page(self, name, **kwargs):
        def decorator(pfunc):
            groups = vod(kwargs,'groups')
            if (groups is None or # Page is available to all
                vod(self.user,'group')=='admin' or # Admin sees all
                vod(self.user,'group','guests') in groups): # group is whitelisted
                self.pages.append( (name,pfunc,kwargs) )
        return decorator

    def build(self):    
        # If login failed and is required, don't go any further
        if not self.public and not st.session_state["authentication_status"]: return
    
        # Add user settings page if logged in
        if st.session_state["authentication_status"]:  self.pages.append( ('Settings',user_settings_page,{'icon': 'sliders'}) )
        
        # Add admin page for admins
        if vod(self.user,'group')=='admin':  self.pages.append( ('Admin',admin_page,{'icon': 'terminal'}) )
        
        # Draw the menu listing pages
        pnames = [t[0] for t in self.pages]
        with st.sidebar:
            
            if st.session_state["authentication_status"]:
                self.sb_info.info(f'Logged in as **{self.user["name"]}**')
                self.auth.logout('Logout', 'sidebar')
            
            menu_choice = option_menu("Pages",
                pnames, icons=[vod(t[2],'icon') for t in self.pages],
                styles={
                    "container": {"padding": "5!important"}, #, "background-color": "#fafafa"},
                    #"icon": {"color": "red", "font-size": "15px"},
                    "nav-link": {"font-size": "12px", "text-align": "left", "margin":"0px", "--hover-color": "#eee"},
                    "nav-link-selected": {"background-color": "#red"},
                    "menu-title": {"display":"none"}
                })
            
        # Render the chosen page
        pname, pfunc, meta = self.pages[pnames.index(menu_choice)]
        st.title(pname)
        pfunc(**clean_kwargs(pfunc,{'sdb':self}))
        
    # Add enter and exit so it can be used as a context
    def __enter__(self):
        return self
    
    # Render everything once we exit the with block
    def __exit__(self, exc_type, exc_value, exc_tb):
        self.build()
    
        

In [ ]:
#| exporti 

def admin_page(sdb):
    st.write('**Under construction**')
    pass

def user_settings_page(sdb):
    if not st.session_state["authentication_status"]: return
    try:
        if sdb.auth.reset_password(st.session_state["username"], 'Reset password'):
            sdb.save_auth_conf()
            st.success('Password modified successfully')
    except Exception as e:
        st.error(e)

## Authentication

In [ ]:
stauth.Hasher(['kalasaba']).generate()

['$2b$12$AEsY9kDn3ENq27uFi3f2tO67BVWRiDqgonS5YqH9oF6x8X3jtCLYq']

## Admin

In [ ]:
# Snippets copied over from dashboard.py to be re-purposed here. 

In [ ]:

        st.subheader('🛠️ Administration')
        st.toast('Connecting to AWS...')

        log_management, userlist, adduser, changeuser, deleteuser = st.tabs([
            'Log management',
            'List users',
            'Add user',
            'Change user',
            'Delete user'
        ])
        st.write(" ")

        with log_management:
            if st.button('View logfile', disabled=restricted):
                log_data=pd.read_csv(log_file)
                st.dataframe(log_data.sort_index(ascending=False
                    ).style.applymap(highlight_cells, subset=['status']), width=1200) #use_container_width=True
            if st.button('Reset logfile', disabled=True): #disabled=restricted
                create_log(log_file)

            #if st.button('Read S3'):
            #    st.write(s3_read_new('salk-test/users.json'))

        with userlist:
            st.markdown(hide_dataframe_row_index, unsafe_allow_html=True)
            st.dataframe(list_users(), use_container_width=True)

        with adduser:
            new_user()

        with changeuser:
            change_user(usernames)

        with deleteuser:
            delete_user()

In [ ]:
def new_user():
    with st.form("add_user_form"):
        st.subheader("Lisa kasutaja:")
        st.markdown("""---""")
        col1,col2 = st.columns((1,2))
        with col1:
            new_group = st.radio("Kasutajagrupp:", ('Külaline', 'Meedia', 'Erakond', 'SALK', 'Admin'))
        with col2:
            new_user = st.text_input("Kasutaja:")
            new_name = st.text_input("Nimi:")
            new_password = st.text_input("Parool:", type='password')
            st.markdown("""---""")
            new_email = st.text_input("E-mail:")
        st.markdown("""---""")
        submitted = st.form_submit_button("Kinnita")
        if submitted:
            if not '' in [new_user, new_password, new_email]:
                add_user(new_user, new_name, new_password, new_group, new_email)
                st.experimental_memo.clear()
                st.experimental_rerun()
            else:
                info.error('Must specify username, password and email.')

def add_user(user, name, password, group, email):
    with fs.open(user_file,'r') as file:
        data = json.load(file)

    if user not in list(data.keys()):
        new_user={}
        new_user['password']=make_hashes(password)
        new_user['name']=name
        new_user['group']=group
        new_user['e-mail']=email
        data[user]=new_user

        with fs.open(user_file, 'w') as outfile:
            json.dump(data, outfile)
            outfile.close()

        info.success('Kasutaja {} lisatud.'.format(user))
        log_event('add-user: ' + user, st.session_state['username'])
        #st.experimental_memo.clear()  #clear cache and rerun
        return True
    else:
        info.error('Kasutaja **{}** on juba olemas.'.format(user))
        return False

def change_password(user):
    with st.form("password_form"):
        st.subheader("Muuda parooli:")
        col1,col2 = st.columns((1,2))
        with col1:
            user = st.text_input("Kasutaja:", value=user, disabled=True)
        with col2:
            pwd = st.text_input("Vana parool:", type='password')
        st.markdown("""---""")
        pwd_1 = st.text_input("Uus parool:", type='password')
        pwd_2 = st.text_input("Veel kord:", type='password')
        if pwd_1 == '':
            vaheta = False
        elif pwd_1 == pwd_2:
            password = pwd_1
            vaheta = True
        else:
            st.error('Salasõnad ei klapi.')
            vaheta = False
        submitted = st.form_submit_button("Muuda")

        if submitted:
            hash = make_hashes(pwd)
            if vaheta:
                with fs.open(user_file, 'r') as f:
                   user_dict=json.load(f)
                   f.close()
                if bcrypt.checkpw(pwd.encode(), user_dict[user]['password'].encode()):
                    user_dict[user]['password'] = make_hashes(password)
                    with fs.open(user_file, 'w') as outfile:
                        json.dump(user_dict, outfile)
                        outfile.close()
                    info.success('Kasutaja **{}** parool uuendatud.'.format(user))
                    log_event('password-change', user)
                    #load_users.clear()
                    #st.experimental_memo.clear()
                    #st.experimental_rerun()
                    return True
                else:
                    info.error('Vale salasõna.')
                    return False

def delete_user():
    with fs.open(user_file, 'r') as f:
        data=f.read()
        u_dict=json.loads(data)

    with st.form("delete_user_form"):
        st.subheader('Delete user:')
        user = st.selectbox('Select username:', list(u_dict.keys()))
        check = st.checkbox('Deletion is FINAL and cannot be undone!')
        st.markdown("""___""")
        submitted = st.form_submit_button("Delete")
        if submitted:
            if check and user == st.session_state['username']:
                info.error('Cannot delete the current user.')
            elif check:
                del u_dict[user]
                with fs.open(user_file, 'w') as outfile:
                    json.dump(u_dict, outfile)
                    outfile.close()
                info.warning('User **{}** deleted.'.format(user))
                log_event('delete-user: ' + user, st.session_state['username'])
                st.experimental_memo.clear()
                st.experimental_rerun()
            else:
                info.warning('Tick the checkbox in order to delete user **{}**.'.format(user))

def change_user(usernames):
    user_groups = ['Külaline', 'Meedia', 'Erakond', 'SALK', 'Admin']
    username=st.selectbox('Muuda kasutajat', usernames)
    with fs.open(user_file, 'r') as f:
        data=f.read()
        u_dict=json.loads(data)
    user_record = u_dict[username]
    #st.write(user_record)
    group_index = user_groups.index(user_record['group'])

    with st.form("edit_user_form"):
        st.subheader("Muuda kasutaja andmeid:")
        st.markdown("""---""")
        col1,col2 = st.columns((1,2))
        with col1:
            user = st.text_input("Kasutaja:", value=username, disabled=True)
            group = st.radio("Kasutajagrupp:", user_groups, index=group_index) #, disabled=True)
        with col2:
            #new_user = st.text_input("Kasutaja:", value=username, disabled=True)
            name = st.text_input("Nimi:", value=user_record['name'])
            password = st.text_input("Parool:", type='password')
            st.markdown("""---""")
            email = st.text_input("E-mail:", value=user_record['e-mail'])
        st.markdown("""---""")
        submitted = st.form_submit_button("Kinnita")
        if submitted:
            with fs.open(user_file, 'r') as f:
                user_dict=json.loads(f.read())
                f.close()
            user_dict[user]['name'] = name
            user_dict[user]['group'] = group
            user_dict[user]['email'] = email
            if password != '':
                user_dict[user]['password'] = make_hashes(password)
            else:
                user_dict[user]['password'] = user_record['password']
            with fs.open(user_file, 'w') as outfile:
                json.dump(user_dict, outfile)
                info.success('Kasutaja {} muudetud.'.format(user))
                outfile.close()

def make_hashes(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()

def check_hashes(password, hashed_text):
    if make_hashes(password) == hashed_text:
        return hashed_text
    return False

def list_users():
    with fs.open(user_file, 'r') as f:
        data=f.read()
        u_dict=json.loads(data)
        f.close()

    log_data=pd.read_csv(log_file)
    log_data['timestamp']= pd.to_datetime(log_data['timestamp'],
        utc=True, infer_datetime_format=True)

    login_list = []
    name_list = []
    email_list = []
    group_list = []

    for u in u_dict.keys():
        last_login=log_data[log_data.user == u].timestamp.max()
        if pd.notnull(last_login):
            last_login=last_login.strftime('%d-%b-%Y')
        login_list.append(last_login)
        name_list.append(u_dict[u]['name'])
        email_list.append(u_dict[u]['e-mail'])
        group_list.append(u_dict[u]['group'])

    d = {
        'user': u_dict.keys(),
        'name': name_list,
        'group': group_list,
        #'e-mail': email_list,
        'last login': login_list
        }

    return pd.DataFrame(d)


# Setup guide
- User conf
  - cookie key matters. generate a decent one randomly

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()